In [1]:
# 載入基本套件
import os
import pandas as pd
import numpy as np
# 模型
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [48]:
# load data
# 檔案路徑
data_path = 'data/kaggle/data'
train_path = os.path.join(data_path, 'train.csv')
train_label_path = os.path.join(data_path, 'trainLabels.csv')
test_path = os.path.join(data_path, 'test.csv')

# 讀取訓練與測試資料
df_train = pd.read_csv(train_path,header=None)
df_test = pd.read_csv(test_path,header=None)
train_Y = pd.read_csv(train_label_path,header=None)
print(df_train.shape)
print(df_test.shape)
print(df_trainLabels.shape)

(1000, 40)
(9000, 40)
(1000, 1)


In [49]:
# 合併資料做特徵工程
df = pd.concat([df_train,df_test],sort=False)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.850465,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.022959,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,0.513906,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332


In [12]:
# 載入標籤編碼與最小最大化, 以便做最小的特徵工程
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# 特徵工程-簡化版 : 全部空值先補-1,數字欄位做 MinMaxScaler(將特徵的取值縮小到一個範圍)(這麼做雖然有些暴力, 卻可以最簡單的平衡特徵間影響力)
MMEncoder = MinMaxScaler()

In [10]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39],
           dtype='int64')

In [13]:
for c in df.columns:
    df[c] = df[c].fillna(-1) # Na 補 -1
    # 將數值欄位最大最小化, 因為需要是一維陣列, 所以這邊切出來後用 reshape 降維
    df[c] = MMEncoder.fit_transform(df[c].values.reshape(-1, 1))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.521848,0.385000,0.696357,0.337709,0.596758,0.459816,0.678536,0.620093,0.514675,0.570771,...,0.452269,0.403072,0.390280,0.563861,0.790045,0.573629,0.698835,0.143881,0.358967,0.462177
1,0.323043,0.584283,0.621256,0.322782,0.515096,0.415239,0.512366,0.687253,0.422438,0.470981,...,0.456028,0.480840,0.616589,0.583676,0.530993,0.534213,0.557137,0.655388,0.624663,0.351963
2,0.642301,0.488843,0.500375,0.193134,0.553666,0.445027,0.549366,0.239082,0.368326,0.319846,...,0.482374,0.571220,0.301001,0.627517,0.388825,0.266564,0.606321,0.582898,0.473336,0.627967
3,0.693710,0.467627,0.372577,0.415962,0.539579,0.541891,0.527920,0.810493,0.655478,0.339302,...,0.681567,0.635581,0.294011,0.410551,0.700972,0.686549,0.668321,0.525522,0.574777,0.564552
4,0.398744,0.459459,0.643525,0.494089,0.549207,0.432513,0.622141,0.234414,0.293894,0.589808,...,0.619261,0.258504,0.527731,0.507645,0.261976,0.274937,0.615291,0.610803,0.410455,0.479827


In [50]:
train_num = train_Y.shape[0]
train_num

1000

In [51]:
# 將前述轉換完畢資料 df , 重新切成 train_X, test_X, 因為不論何種特徵工程, 都需要對 train / test 做同樣處理
# 常見並簡便的方式就是 - 先將 train / test 接起來, 做完後再拆開, 不然過程當中往往需要將特徵工程部分寫兩次, 麻煩且容易遺漏
# 在較複雜的特徵工程中尤其如此, 若實務上如果碰到 train 與 test 需要分階段進行, 則通常會另外寫成函數處理
train_X = df[:train_num]
test_X = df[train_num:]

In [57]:
# 切分訓練集/測試集
x_train, x_valid, y_train, y_valid = train_test_split(train_X, df_trainLabels, test_size=0.2, random_state=4)

# 建立模型
clf = GradientBoostingClassifier(random_state=7)

# 將訓練資料丟進去模型訓練(參數default)
clf.fit(x_train, y_train)

# 將測試資料丟進模型得到預測結果
y_pred = clf.predict(x_valid)

# 先看看使用預設參數得到的結果，約為 0.86 的 accuracy
print(metrics.accuracy_score(y_valid, y_pred))

D:\software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.86


In [58]:
# 建立模型
clf = GradientBoostingClassifier(
    random_state=7,
    learning_rate=0.05
)

In [59]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300] #default = 100
max_depth = [1, 5, 7 ,10] #default = 3
min_samples_leaf = [3,5,7]#default = 1
min_samples_split = [3,4,5] #default = 2
param_grid = dict(n_estimators=n_estimators, 
                  max_depth=max_depth,
                  min_samples_leaf=min_samples_leaf,
                  min_samples_split=min_samples_split)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:  2.5min finished
D:\software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [60]:
# 印出最佳結果與最佳參數
print("Best accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best accuracy: 0.887500 using {'max_depth': 5, 'min_samples_leaf': 7, 'min_samples_split': 3, 'n_estimators': 200}


In [61]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(
    random_state=7,
    learning_rate=0.05,
    max_depth=grid_result.best_params_['max_depth'],
    n_estimators=grid_result.best_params_['n_estimators'],
    min_samples_leaf=grid_result.best_params_['min_samples_leaf'],
    min_samples_split=grid_result.best_params_['min_samples_split'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_valid)

# 調整參數後約可提升至 0.9的 accuracy
print(metrics.accuracy_score(y_valid, y_pred))

D:\software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9


In [67]:
# 預測測試集
y_pred = clf_bestparam.predict(test_X)
submit = pd.DataFrame({'Id': np.arange(9000)+1, 'Solution': y_pred})
submit.to_csv('D48HW_sklearn_practice_submit_GBclassifier.csv', index=False)